In [ ]:
#On my mac, it was a real pain to get the comma2k19 dataset to work. I had to do the following but you might not have to:

!pip uninstall lru-dict -y
!ARCHFLAGS="-arch arm64" pip install lru-dict --compile --no-cache-dir
!set -x PYCURL_SSL_LIBRARY openssl
!export PYCURL_SSL_LIBRARY=openssl
!export LDFLAGS=-L/usr/local/opt/openssl/lib
!export CPPFLAGS=-I/usr/local/opt/openssl/include
!ARCHFLAGS="-arch arm64" pip install pycurl --no-cache-dir
!arch -arm64 pip install pycurl --compile --no-cache-dir
!ARCHFLAGS="-arch arm64" pip install pycapnp
!brew uninstall openssl
!brew install https://github.com/tebelorg/Tump/releases/download/v1.0.0/openssl.rb

In [1]:
from __future__ import print_function
import os
import numpy as np
import sys
from tqdm import tqdm
import cv2
import platform
platform.architecture()
sys.path.append("../openpilot")
from tools.lib.logreader import LogReader
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tools.lib.framereader import FrameReader
import h5py


In [2]:
example_segment = '../Example_1/b0c9d2329ad1606b|2018-08-02--08-34-47/40/'
lr = LogReader(example_segment + 'raw_log.bz2')
# make list of logs
logs = list(lr)
[l.which() for l in logs[:50]]

['sensorEventsDEPRECATED',
 'controlsState',
 'sendcan',
 'carState',
 'carControl',
 'logMessage',
 'can',
 'longitudinalPlan',
 'can',
 'sendcan',
 'sensorEventsDEPRECATED',
 'sensorEventsDEPRECATED',
 'carState',
 'controlsState',
 'carControl',
 'logMessage',
 'logMessage',
 'logMessage',
 'can',
 'roadEncodeIdx',
 'roadEncodeIdx',
 'longitudinalPlan',
 'controlsState',
 'sendcan',
 'carState',
 'carControl',
 'sensorEventsDEPRECATED',
 'can',
 'logMessage',
 'logMessage',
 'radarState',
 'liveTracks',
 'can',
 'sensorEventsDEPRECATED',
 'liveLongitudinalMpcDEPRECATED',
 'can',
 'longitudinalPlan',
 'liveLongitudinalMpcDEPRECATED',
 'sendcan',
 'sensorEventsDEPRECATED',
 'sensorEventsDEPRECATED',
 'sensorEventsDEPRECATED',
 'controlsState',
 'carState',
 'carControl',
 'can',
 'roadCameraState',
 'sensorEventsDEPRECATED',
 'sensorEventsDEPRECATED',
 'sensorEventsDEPRECATED']

In [5]:
# We can extract frames from the video with framereader
# from openpilot tools, we look at frame 600
frame_index = 600

fr = FrameReader(example_segment + 'video.hevc')
#figsize(12,12)
#imshow(fr.get(frame_index, pix_fmt='rgb24')[0]);
#title('Frame 600 extracted from video with FrameReader', fontsize=25);

In [67]:
def get_sample(p):
    frame_reader = FrameReader(p+'/video.hevc')
    logs = list(LogReader(p + '/raw_log.bz2'))

    angle = np.array([l.carState.steeringAngleDeg for l in logs if l.which() == 'carState'])[1::5][1::5]
    time = np.array([l.logMonoTime for l in logs if l.which() == 'carState'])[1::5][1::5]
    vEgo = np.array([l.carState.vEgo for l in logs if l.which() == 'carState'])[1::5][1::5]
    gas = np.array([l.carState.gas for l in logs if l.which() == 'carState'])[1::5][1::5]
    brake = np.array([l.carState.brake for l in logs if l.which() == 'carState'])[1::5][1::5]
    gps_times = np.load(p + '/global_pose/frame_gps_times')
    times = np.load(p + '/global_pose/frame_times')
    gas = np.array([l.carState.gas for l in logs if l.which() == 'carState'])[1::5][1::5]
    gaspressed = np.array([l.carState.gasPressed for l in logs if l.which() == 'carState'])[1::5][1::5]
    brake = np.array([l.carState.brake for l in logs if l.which() == 'carState'])[1::5][1::5]
    brake_pressed = np.array([l.carState.brakePressed for l in logs if l.which() == 'carState'])[1::5][1::5]

    enabled = np.array([l.carState.cruiseState.enabled for l in logs if l.which() == 'carState'])[1::5][1::5]
    speed = np.array([l.carState.cruiseState.speed for l in logs if l.which() == 'carState'])[1::5][1::5]
    speedOffset = np.array([l.carState.cruiseState.speedOffset for l in logs if l.which() == 'carState'])[1::5][1::5]
    standstill = np.array([l.carState.cruiseState.standstill for l in logs if l.which() == 'carState'])[1::5][1::5]
    nonAdaptive = np.array([l.carState.cruiseState.nonAdaptive for l in logs if l.which() == 'carState'])[1::5][1::5]
    speedCluster = np.array([l.carState.cruiseState.speedCluster for l in logs if l.which() == 'carState'])[1::5][1::5]
    
    leftBlinker = np.array([l.carState.leftBlinker for l in logs if l.which() == 'carState'])[1::5][1::5]
    rightBlinker = np.array([l.carState.rightBlinker for l in logs if l.which() == 'carState'])[1::5][1::5]
    #print(frame_reader.frame_count, gps_times.shape, times.shape)
    #print([l.carState for l in logs if l.which() == "carState"][0])
    #print([l.radarState for l in logs if l.which() == "radarState"][0])

    dist = np.array([l.radarState.leadOne.dRel for l in logs if l.which() == "radarState"])[1::5]
    if ((vEgo == 0).mean() > 0.2) or ((dist == 0).mean() > 0.2) or len(dist) <=230:
        return None
    images = []
    l = list(range(frame_reader.frame_count))
    if len(l) > 245:
        l = l[1::5]
    for idx in list(range(frame_reader.frame_count))[1::5]:
        image = np.array(frame_reader.get(idx, pix_fmt='rgb24')[0], dtype=np.float64)
        image = cv2.resize(image, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        images.append(image)
    steady_state = ~gaspressed & ~brake_pressed & ~leftBlinker & ~rightBlinker
    last_idx = 0
    desired_gap = np.zeros(steady_state.shape)

    for i in range(len(steady_state)-1):
        if steady_state[i] == True:
            desired_gap[last_idx:i] = int(dist[i])
            last_idx = i

    sample = {
        'image': images,
        "CruiseStateenabled": enabled, 
        "CruiseStatespeed": speed, 
        "CruiseStatespeedOffset": speedOffset,
        "CruiseStatestandstill": standstill, 
        "CruiseStatenonAdaptive": nonAdaptive, 
        "CruiseStatespeedCluster": speedCluster, 
        'leftBlinker': leftBlinker, 
        'rightBlinker': rightBlinker, 
        "gas": gas, 
        "gaspressed": gaspressed, 
        "brake": brake, 
        "brakepressed": brake_pressed, 
        'angle': angle, 
        'time': time, 
        'gas': gas, 
        'vEgo': vEgo, 
        'brake': brake, 
        'dist': dist, 
        'desired_dist': desired_gap,
        } 
    return sample if not ((desired_gap == 0).mean() > 0.2) else None

In [68]:
def save_h5py(i, sample, h):
    group = h.create_group(str(i))
    for col in sample.keys():
            dt = np.float32 if col != 'image' else int#
            dataset_name = col #groups are divided by '/'
            a = list(sample[col])
            group.create_dataset(dataset_name, data = np.asarray(a, dtype=dt),
                    #compression_opts=9,
                    #chunks=(164, 20, 20, 3),
                    compression='lzf')

In [74]:
dataset_type = "train"
main_dir='/Users/jessicaechterhoff/Dropbox/UCSD_2022_2023/Research/toyota/commaai/comma2k19/comma2k19/Chunk_1/'
hdf5_filename = "gas_and_brake_train_comma_chunk_1_w_imgs.hfd5"
h_train = h5py.File(hdf5_filename, 'w')
hdf5_filename = "gas_and_brake_val_comma_chunk_1_w_imgs.hfd5"
h_val = h5py.File(hdf5_filename, 'w')
hdf5_filename = "gas_and_brake_test_comma_chunk_1_w_imgs.hfd5"
h_test = h5py.File(hdf5_filename, 'w')

In [75]:
for j, drive_sequence_path in tqdm(enumerate(os.listdir(main_dir))):
    if '.DS_Store ' in drive_sequence_path or not os.path.isdir(main_dir+"/"+drive_sequence_path): continue
    min_sequence_paths = os.listdir(main_dir+"/"+drive_sequence_path)
    if len(min_sequence_paths) < 3: continue
    min_sequence_path_test = main_dir+"/"+drive_sequence_path+"/"+min_sequence_paths[-2]
    min_sequence_paths_val = main_dir+"/"+drive_sequence_path+"/"+min_sequence_paths[-1]
    sample = get_sample(min_sequence_path_test)
    if sample != None:
        save_h5py(f"{drive_sequence_path}", sample, h_test)
    sample = get_sample(min_sequence_paths_val)

    if sample != None:
        save_h5py(f"{drive_sequence_path}", sample, h_val)
    for i, min_sequence in enumerate(min_sequence_paths[:-2]):
        if '.DS_Store ' in min_sequence or not os.path.isdir(main_dir+"/"+drive_sequence_path+'/'+min_sequence): continue
        p = main_dir+"/"+drive_sequence_path+"/"+min_sequence
        sample = get_sample(p)
        if sample != None:
            save_h5py(f"{drive_sequence_path}_{min_sequence}", sample, h_train)
    

23it [28:05, 73.28s/it] 


In [77]:
h_test.close()
h_train.close()
h_val.close()

Get data on how much cruise state is enabled

In [20]:
with h5py.File('gas_and_brake_train_comma_chunk_1_w_imgs.hfd5', "r") as f:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    #     # get first object name/key; may or may NOT be a group
    all_group_key = list(f.keys())
    allcc = []
    for a_group_key in all_group_key:
    
        data = list(f[a_group_key])
        # preferred methods to get dataset values:
        ds_obj = f[a_group_key]      # returns as a h5py dataset object
        ds_arr = f[a_group_key]['dist'][()]
        ds_arr = f[a_group_key]['CruiseStateenabled'][()].mean()
        allcc.append(ds_arr)
    print(np.array(allcc).mean())

0.6902778


In [21]:
with h5py.File('gas_and_brake_train_comma_chunk_3_w_imgs.hfd5', "r") as f:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    #     # get first object name/key; may or may NOT be a group
    all_group_key = list(f.keys())
    allcc = []
    for a_group_key in all_group_key:
    
        data = list(f[a_group_key])
        # preferred methods to get dataset values:
        ds_obj = f[a_group_key]      # returns as a h5py dataset object
        ds_arr = f[a_group_key]['dist'][()]
        ds_arr = f[a_group_key]['CruiseStateenabled'][()].mean()
        allcc.append(ds_arr)
    print(np.array(allcc).mean())

0.6369225


In [19]:
with h5py.File('gas_and_brake_train_comma_chunk_2_w_imgs.hfd5', "r") as f:
    # Print all root level object names (aka keys) 
    # these can be group or dataset names 
    #     # get first object name/key; may or may NOT be a group
    all_group_key = list(f.keys())
    allcc = []
    for a_group_key in all_group_key:
    
        data = list(f[a_group_key])
        # preferred methods to get dataset values:
        ds_obj = f[a_group_key]      # returns as a h5py dataset object
        ds_arr = f[a_group_key]['dist'][()]
        ds_arr = f[a_group_key]['CruiseStateenabled'][()].mean()
        allcc.append(ds_arr)
    print(np.array(allcc).mean())

0.8325231
